In [23]:
from pathlib import Path
import pandas as pd
import numpy as np

# Vehicle registration data cleaning

In [ ]:
#This was performed in Excel.
# cleaned file is called 'Cleaned_Postcode-Registrations-Data-1 (1).xlsx'


# Census Data cleaning

First clean - remove unnecessary columns, remove the extra data before the ':' in each column and rename the columns

In [2]:
#define the path for reading files
ABS_data = Path('Resources/A1_copy.csv')
#read in csv to datafile
obs_df = pd.read_csv(ABS_data)
obs_df.head()

,DATAFLOW,SEXP: Sex,INCP: Total personal income (weekly),AGEP: Age,REGION: Region,REGION_TYPE: Region Type,STATE: State,TIME_PERIOD: Time Period,OBS_VALUE
0,ABS:C21_G17_POA(1.0.0),3: Persons,"12: $1,750-$1,999",55_64: 55-64 years,3856: 3856,POA: Postal Areas,2: Victoria,2021,10
1,ABS:C21_G17_POA(1.0.0),3: Persons,"14: $3,000-$3499",15_19: 15-19 years,3150: 3150,POA: Postal Areas,2: Victoria,2021,0
2,ABS:C21_G17_POA(1.0.0),3: Persons,"13: $2,000-$2,999",45_54: 45-54 years,3196: 3196,POA: Postal Areas,2: Victoria,2021,578
3,ABS:C21_G17_POA(1.0.0),3: Persons,"12: $1,750-$1,999",GE85: 85 years and over,3249: 3249,POA: Postal Areas,2: Victoria,2021,0
4,ABS:C21_G17_POA(1.0.0),3: Persons,"14: $3,000-$3499",_T: Total,3312: 3312,POA: Postal Areas,2: Victoria,2021,13


In [3]:
#remove the extra columns using column indices
columns_to_keep_indices = [2,3,4,6,8]
obs_df = obs_df.iloc[:, columns_to_keep_indices]
#remove extra values in columns using indices, note only first 5 columns need cleaning
columns_to_clean = obs_df.columns[:4]
# Apply the split operation to the specified columns
obs_df[columns_to_clean] = obs_df[columns_to_clean].apply(lambda x: x.apply(lambda y: str(y).split(':')[1]))
# Apply new column names
Col_names = ['Total personal income (weekly)', 'Age', 'Postcode', 'State', 'Population']
obs_df.columns = Col_names

# Save the modified DataFrame back to a new CSV file 
obs_df.to_csv('Resources/Census_clean.csv', index=False)
obs_df.head()

,Total personal income (weekly),Age,Postcode,State,Population
0,"$1,750-$1,999",55-64 years,3856,Victoria,10
1,"$3,000-$3499",15-19 years,3150,Victoria,0
2,"$2,000-$2,999",45-54 years,3196,Victoria,578
3,"$1,750-$1,999",85 years and over,3249,Victoria,0
4,"$3,000-$3499",Total,3312,Victoria,13


Adding mid point to Census Age and Income ranges

In [4]:
#create a file that has Midpoint Age and Midpoint income

#Define the function to convert the Income ranges to midpoint
def convert_range_to_midpoint(value):
    value = value.replace('$', '').replace(',', '')  # Remove dollar signs and commas
    if 'Total' in value or 'Not stated' in value or 'Negative/Nil income' in value:
        return 0  # Set a default value for 'Total', 'Not stated', or 'Negative/Nil income'
    elif 'or more' in value:
        return '3500+'  # Set a special value for '3500 or more'
    elif '-' in value:
        return sum(map(int, value.split('-'))) / 2
    else:
        return int(value)
#apply function to Total Personal Income and create a new column
obs_df['Income_Midpoint'] = obs_df['Total personal income (weekly)'].apply(convert_range_to_midpoint)

obs_df.head()
 

,Total personal income (weekly),Age,Postcode,State,Population,Income_Midpoint
0,"$1,750-$1,999",55-64 years,3856,Victoria,10,1874.5
1,"$3,000-$3499",15-19 years,3150,Victoria,0,3249.5
2,"$2,000-$2,999",45-54 years,3196,Victoria,578,2499.5
3,"$1,750-$1,999",85 years and over,3249,Victoria,0,1874.5
4,"$3,000-$3499",Total,3312,Victoria,13,3249.5


In [5]:
# Define the function to convert age to midpoint
def convert_age_to_midpoint(value):
    if 'Total' in value:
        return None  # Ignore 'Total'
    
    if 'years' in value:
        value = value.replace(' years', '')
        if '-' in value:
            return sum(map(int, value.split('-'))) / 2  # Midpoint for ranges like '55-64'
        else:
            return '85+'  # Special value for '85 years and over'
    
    return 0  # Default value

# Convert 'Age' column and create a new column 'Age_Midpoint'
obs_df['Age_Midpoint'] = obs_df['Age'].apply(convert_age_to_midpoint)
# Remove the rows where Age Midpoint is None
obs_df = obs_df.dropna(subset=['Age_Midpoint'])

obs_df.head()


,Total personal income (weekly),Age,Postcode,State,Population,Income_Midpoint,Age_Midpoint
0,"$1,750-$1,999",55-64 years,3856,Victoria,10,1874.5,59.5
1,"$3,000-$3499",15-19 years,3150,Victoria,0,3249.5,17.0
2,"$2,000-$2,999",45-54 years,3196,Victoria,578,2499.5,49.5
3,"$1,750-$1,999",85 years and over,3249,Victoria,0,1874.5,85+
5,"$2,000-$2,999",35-44 years,3378,Victoria,5,2499.5,39.5


In [6]:
#save this new file
obs_df.to_csv('Resources/Census_clean_Midpoints_added.csv', index=False)

## Creating the Top 10 Fuel type by Registration

Grouping and finding the top 10 for each year

In [ ]:
# Load the Excel file
file_path = 'Resources/Cleaned_Postcode-Registrations-Data-1 (1).xlsx'
data = pd.read_excel(file_path)
# Display the first few rows of the dataframe
data.head()

In [ ]:
# Grouping and finding top 10 for each year

# For the year 2021
top_2021 = data.groupby(['Fuel Type', 'Postcode','State'])['Registrations as at 31 January 2021'].sum().reset_index()
top_2021 = top_2021.sort_values(by='Registrations as at 31 January 2021', ascending=False).groupby('Fuel Type').head(10)
# Exclude the HFCEV fuel type as it is all zeros
top_2021 = top_2021[top_2021['Fuel Type'] != 'HFCEV']
# For the year 2022
top_2022 = data.groupby(['Fuel Type', 'Postcode','State'])['Registrations as at 31 January 2022'].sum().reset_index()
top_2022 = top_2022.sort_values(by='Registrations as at 31 January 2022', ascending=False).groupby('Fuel Type').head(10)
# Exclude the HFCEV fuel type as it is all zeros
top_2022 = top_2022[top_2022['Fuel Type'] != 'HFCEV']
# For the year 2023
top_2023 = data.groupby(['Fuel Type', 'Postcode','State'])['Registrations as at 31 January 2023'].sum().reset_index()
top_2023 = top_2023.sort_values(by='Registrations as at 31 January 2023', ascending=False).groupby('Fuel Type').head(10)
# Exclude the HFCEV fuel type as it is all zeros
top_2023 = top_2023[top_2023['Fuel Type'] != 'HFCEV']

top_2021, top_2022, top_2023


In [ ]:
# Adding a year column to each DataFrame
top_2021['Year'] = 2021
top_2022['Year'] = 2022
top_2023['Year'] = 2023

# Renaming the registration columns
top_2021.rename(columns={'Registrations as at 31 January 2021': 'Registrations'}, inplace=True)
top_2022.rename(columns={'Registrations as at 31 January 2022': 'Registrations'}, inplace=True)
top_2023.rename(columns={'Registrations as at 31 January 2023': 'Registrations'}, inplace=True)

# Concatenating the DataFrames
combined_df = pd.concat([top_2021, top_2022, top_2023])

In [ ]:
#Save the file 
combined_df.to_csv('Resources/Top_10_for_all_years.csv', index=False)

# Creating seperate files for each vehicle type

In [9]:
#read in the datafile containing the fuel type and postcode
top10_for2021_df = pd.read_csv('Resources/Cleaned_post_code_registration.csv')

#remove the extra columns using column indices
columns_to_keep_indices = [0,1,2,4]
top10_for2021_df = top10_for2021_df.iloc[:, columns_to_keep_indices]

#Sort the df based on number of registrations
sorted_df = top10_for2021_df.sort_values(by= 'Registrations as at 31 January 2022', ascending=False)

#Group by the fuel type
grouped_df = sorted_df.groupby('Fuel Type')

#Create a dictionary to store each fuel type
fuel_type_ds = {}
#for loop to get the top 10 and store in dictionary, also dropping Hydrogen Cell Vehicle.
for fuel_type, group in grouped_df:
    if fuel_type != 'Hydrogen Cell Vehicle':
        fuel_type_ds[fuel_type] = group.head(10).reset_index(drop=True)


In [24]:
resource_path = Path('Resources')
#read in the income data to merge with the above dictionary
income_df = pd.read_csv('Resources/Census_clean_Midpoints_added.csv')

#create an empty dictionary for the merge
merged_ds = {}

#for loop to merge the datasets
for fuel_type, fuel_type_df in fuel_type_ds.items():
    merged_df = pd.merge(fuel_type_df, income_df, on='Postcode', how ='left')
    #filter rows to remove Population of 0
    merged_df = merged_df[merged_df['Population'] !=0]
    merged_ds[fuel_type]= merged_df
    #Drop extra state column
    merged_df = merged_df.drop('State_y', axis=1)
    #rename remaining state column
    merged_df.rename(columns={'State_x':'State'}, inplace=True)
    #Save the output to a csv file for each fuel type
    filename = resource_path / f'{fuel_type}_top10.csv'
    merged_df.to_csv(filename, index=False)
    #add print statement to show named files created
    print(f'Saved {fuel_type} merge to {filename}')
    



Saved Battery Electric Vehicle merge to Resources/Battery Electric Vehicle_top10.csv
Saved Hybrid merge to Resources/Hybrid_top10.csv
Saved Internal Combustion Engine merge to Resources/Internal Combustion Engine_top10.csv


In [25]:
merge = pd.read_csv('Resources/Battery Electric Vehicle_top10.csv')
merge.head()

,Postcode,State,Fuel Type,Registrations as at 31 January 2022,Total personal income (weekly),Age,Population,Income_Midpoint,Age_Midpoint
0,3170,VIC,Battery Electric Vehicle,227,$500-$649,65-74 years,180,574.5,69.5
1,3170,VIC,Battery Electric Vehicle,227,"$2,000-$2,999",25-34 years,220,2499.5,29.5
2,3170,VIC,Battery Electric Vehicle,227,"$1,500-$1,749",75-84 years,6,1624.5,79.5
3,3170,VIC,Battery Electric Vehicle,227,Negative/Nil income,55-64 years,238,0,59.5
4,3170,VIC,Battery Electric Vehicle,227,$800-$999,20-24 years,111,899.5,22.0


In [26]:
#read in the postcode to suburb file
Suburb_df = pd.read_csv('Resources/australian_postcodes.csv')
Suburb_df = Suburb_df.rename(columns={'postcode': 'Postcode'})
Suburb_df['Postcode'] = Suburb_df['Postcode'].drop_duplicates()

# Manipulate the data in the Battery Electric Vehicle file to produce age distribution and income distributions

In [27]:
#read in the top 10 file
csv_file_path = 'Resources/Battery Electric Vehicle_top10.csv'
BEV_df = pd.read_csv(csv_file_path)


In [40]:
# merge Surburb_df with BEV data
merge_BEV = pd.merge(BEV_df, Suburb_df, on='Postcode', how='inner')
columns_to_keep = ['Postcode', 'State', 'Fuel Type','Registrations as at 31 January 2022','Total personal income (weekly)', 'Age', 'Population', 'Income_Midpoint', 'Age_Midpoint', 'locality']
final_merge = merge_BEV[columns_to_keep]
#reorder the columns
desired_order = ['Postcode', 'locality','State', 'Registrations as at 31 January 2022', 'Population', 'Total personal income (weekly)','Income_Midpoint', 'Age', 'Age_Midpoint','Fuel Type']
# new df with reorder
merge_df_reordered = final_merge[desired_order]

merge_df_reordered
#print(merge_BEV.columns)

,Postcode,locality,State,Registrations as at 31 January 2022,Population,Total personal income (weekly),Income_Midpoint,Age,Age_Midpoint,Fuel Type
0,3170,MULGRAVE,VIC,227,180,$500-$649,574.5,65-74 years,69.5,Battery Electric Vehicle
1,3170,MULGRAVE,VIC,227,220,"$2,000-$2,999",2499.5,25-34 years,29.5,Battery Electric Vehicle
2,3170,MULGRAVE,VIC,227,6,"$1,500-$1,749",1624.5,75-84 years,79.5,Battery Electric Vehicle
3,3170,MULGRAVE,VIC,227,238,Negative/Nil income,0,55-64 years,59.5,Battery Electric Vehicle
4,3170,MULGRAVE,VIC,227,111,$800-$999,899.5,20-24 years,22.0,Battery Electric Vehicle
...,...,...,...,...,...,...,...,...,...,...
1473,2602,AINSLIE,ACT,157,958,"$2,000-$2,999",2499.5,25-34 years,29.5,Battery Electric Vehicle
1474,2602,AINSLIE,ACT,157,213,"$1,000-$1,249",1124.5,45-54 years,49.5,Battery Electric Vehicle
1475,2602,AINSLIE,ACT,157,172,$500-$649,574.5,35-44 years,39.5,Battery Electric Vehicle
1476,2602,AINSLIE,ACT,157,139,Negative/Nil income,0,20-24 years,22.0,Battery Electric Vehicle


In [52]:
#Filter the rows where Income is 'Total'
BEV_subset = merge_df_reordered[merge_df_reordered['Total personal income (weekly)'].str.contains('Total', case=False, na=False)]
#Sort by registrations and then age group
BEV_subset = BEV_subset.sort_values(by=['Registrations as at 31 January 2022', 'Age'], ascending=[False, True])

subset_to_save = 'Resources/BEV_top10_total.csv'
BEV_subset.to_csv(subset_to_save, index=False)
BEV_subset

,Postcode,locality,State,Registrations as at 31 January 2022,Population,Total personal income (weekly),Income_Midpoint,Age,Age_Midpoint,Fuel Type
94,4350,ATHOL,QLD,87602,7608,Total,0,15-19 years,17.0,Internal Combustion Engine
147,4350,ATHOL,QLD,87602,7849,Total,0,20-24 years,22.0,Internal Combustion Engine
7,4350,ATHOL,QLD,87602,15863,Total,0,25-34 years,29.5,Internal Combustion Engine
138,4350,ATHOL,QLD,87602,13656,Total,0,35-44 years,39.5,Internal Combustion Engine
50,4350,ATHOL,QLD,87602,13213,Total,0,45-54 years,49.5,Internal Combustion Engine
...,...,...,...,...,...,...,...,...,...,...
1425,6065,ASHBY,WA,57860,9414,Total,0,45-54 years,49.5,Internal Combustion Engine
1343,6065,ASHBY,WA,57860,6672,Total,0,55-64 years,59.5,Internal Combustion Engine
1476,6065,ASHBY,WA,57860,4788,Total,0,65-74 years,69.5,Internal Combustion Engine
1438,6065,ASHBY,WA,57860,2558,Total,0,75-84 years,79.5,Internal Combustion Engine


# Manipulate the data in the Hybrid Vehicle file to produce age distribution and income distributions

In [43]:
#read in the file
csv_file_path = 'Resources/Hybrid_top10.csv'
Hybrid_df = pd.read_csv(csv_file_path)

In [48]:
# merge Surburb_df with Hybrid data
merge_Hybrid = pd.merge(Hybrid_df, Suburb_df, on='Postcode', how='inner')
columns_to_keep = ['Postcode', 'State', 'Fuel Type','Registrations as at 31 January 2022','Total personal income (weekly)', 'Age', 'Population', 'Income_Midpoint', 'Age_Midpoint', 'locality']
final_merge = merge_Hybrid[columns_to_keep]
#reorder the columns
desired_order = ['Postcode', 'locality','State', 'Registrations as at 31 January 2022', 'Population', 'Total personal income (weekly)','Income_Midpoint', 'Age', 'Age_Midpoint','Fuel Type']
# new df with reorder
merge_df_reordered = final_merge[desired_order]

merge_df_reordered


,Postcode,locality,State,Registrations as at 31 January 2022,Population,Total personal income (weekly),Income_Midpoint,Age,Age_Midpoint,Fuel Type
0,4000,BRISBANE,QLD,2511,255,"$3,500 or more",3500+,35-44 years,39.5,Hybrid
1,4000,BRISBANE,QLD,2511,21,$800-$999,899.5,15-19 years,17.0,Hybrid
2,4000,BRISBANE,QLD,2511,125,Negative/Nil income,0,45-54 years,49.5,Hybrid
3,4000,BRISBANE,QLD,2511,127,"$1,750-$1,999",1874.5,45-54 years,49.5,Hybrid
4,4000,BRISBANE,QLD,2511,4,"$2,000-$2,999",2499.5,85 years and over,85+,Hybrid
...,...,...,...,...,...,...,...,...,...,...
1371,4350,ATHOL,QLD,1183,7849,Total,0,20-24 years,22.0,Hybrid
1372,4350,ATHOL,QLD,1183,408,"$3,500 or more",3500+,35-44 years,39.5,Hybrid
1373,4350,ATHOL,QLD,1183,3283,Total,0,85 years and over,85+,Hybrid
1374,4350,ATHOL,QLD,1183,516,$400-$499,449.5,20-24 years,22.0,Hybrid


In [46]:
#Filter the rows where Income is 'Total'
Hybrid_subset = merge_df_reordered[merge_df_reordered['Total personal income (weekly)'].str.contains('Total', case=False, na=False)]
#Sort by registrations and then age group
Hybrid_subset = Hybrid_subset.sort_values(by=['Registrations as at 31 January 2022', 'Age'], ascending=[False, True])
subset_to_save = 'Resources/Hybrid_top10_total.csv'
Hybrid_subset.to_csv(subset_to_save, index=False)
Hybrid_subset

,Postcode,locality,State,Registrations as at 31 January 2022,Population,Total personal income (weekly),Income_Midpoint,Age,Age_Midpoint,Fuel Type
6,4000,BRISBANE,QLD,2511,1093,Total,0,15-19 years,17.0,Hybrid
90,4000,BRISBANE,QLD,2511,3067,Total,0,20-24 years,22.0,Hybrid
64,4000,BRISBANE,QLD,2511,6378,Total,0,25-34 years,29.5,Hybrid
36,4000,BRISBANE,QLD,2511,3228,Total,0,35-44 years,39.5,Hybrid
133,4000,BRISBANE,QLD,2511,2047,Total,0,45-54 years,49.5,Hybrid
...,...,...,...,...,...,...,...,...,...,...
1274,4350,ATHOL,QLD,1183,13213,Total,0,45-54 years,49.5,Hybrid
1335,4350,ATHOL,QLD,1183,12714,Total,0,55-64 years,59.5,Hybrid
1345,4350,ATHOL,QLD,1183,11584,Total,0,65-74 years,69.5,Hybrid
1240,4350,ATHOL,QLD,1183,7612,Total,0,75-84 years,79.5,Hybrid


Manipulate the data in the Internal Combustion Engine Vehicle file to produce age distribution and income distributions

In [49]:
#read in the file
csv_file_path = 'Resources/Internal Combustion Engine_top10.csv'
ICE_df = pd.read_csv(csv_file_path)


In [50]:
# merge Surburb_df with ICE data
merge_Hybrid = pd.merge(ICE_df, Suburb_df, on='Postcode', how='inner')
columns_to_keep = ['Postcode', 'State', 'Fuel Type','Registrations as at 31 January 2022','Total personal income (weekly)', 'Age', 'Population', 'Income_Midpoint', 'Age_Midpoint', 'locality']
final_merge = merge_Hybrid[columns_to_keep]
#reorder the columns
desired_order = ['Postcode', 'locality','State', 'Registrations as at 31 January 2022', 'Population', 'Total personal income (weekly)','Income_Midpoint', 'Age', 'Age_Midpoint','Fuel Type']
# new df with reorder
merge_df_reordered = final_merge[desired_order]

merge_df_reordered


,Postcode,locality,State,Registrations as at 31 January 2022,Population,Total personal income (weekly),Income_Midpoint,Age,Age_Midpoint,Fuel Type
0,4350,ATHOL,QLD,87602,212,"$1,750-$1,999",1874.5,65-74 years,69.5,Internal Combustion Engine
1,4350,ATHOL,QLD,87602,652,Negative/Nil income,0,65-74 years,69.5,Internal Combustion Engine
2,4350,ATHOL,QLD,87602,2075,$800-$999,899.5,25-34 years,29.5,Internal Combustion Engine
3,4350,ATHOL,QLD,87602,78,"$1,000-$1,249",1124.5,15-19 years,17.0,Internal Combustion Engine
4,4350,ATHOL,QLD,87602,167,$1-$149,75.0,45-54 years,49.5,Internal Combustion Engine
...,...,...,...,...,...,...,...,...,...,...
1482,6065,ASHBY,WA,57860,193,Not stated,0,75-84 years,79.5,Internal Combustion Engine
1483,6065,ASHBY,WA,57860,4159,Total,0,20-24 years,22.0,Internal Combustion Engine
1484,6065,ASHBY,WA,57860,269,$150-$299,224.5,25-34 years,29.5,Internal Combustion Engine
1485,6065,ASHBY,WA,57860,915,Total,0,85 years and over,85+,Internal Combustion Engine


In [51]:
#Filter the rows where Income is 'Total'
ICE_subset = merge_df_reordered[merge_df_reordered['Total personal income (weekly)'].str.contains('Total', case=False, na=False)]
#Sort by registrations and then age group
ICE_subset = ICE_subset.sort_values(by=['Registrations as at 31 January 2022', 'Age'], ascending=[False, True])
subset_to_save = 'Resources/ICE_top10_total.csv'
ICE_subset.to_csv(subset_to_save, index=False)
ICE_subset

,Postcode,locality,State,Registrations as at 31 January 2022,Population,Total personal income (weekly),Income_Midpoint,Age,Age_Midpoint,Fuel Type
94,4350,ATHOL,QLD,87602,7608,Total,0,15-19 years,17.0,Internal Combustion Engine
147,4350,ATHOL,QLD,87602,7849,Total,0,20-24 years,22.0,Internal Combustion Engine
7,4350,ATHOL,QLD,87602,15863,Total,0,25-34 years,29.5,Internal Combustion Engine
138,4350,ATHOL,QLD,87602,13656,Total,0,35-44 years,39.5,Internal Combustion Engine
50,4350,ATHOL,QLD,87602,13213,Total,0,45-54 years,49.5,Internal Combustion Engine
...,...,...,...,...,...,...,...,...,...,...
1425,6065,ASHBY,WA,57860,9414,Total,0,45-54 years,49.5,Internal Combustion Engine
1343,6065,ASHBY,WA,57860,6672,Total,0,55-64 years,59.5,Internal Combustion Engine
1476,6065,ASHBY,WA,57860,4788,Total,0,65-74 years,69.5,Internal Combustion Engine
1438,6065,ASHBY,WA,57860,2558,Total,0,75-84 years,79.5,Internal Combustion Engine


## Creating the combined datafiles for plotting

In [ ]:
# Load the new CSV file
income_data_path = 'Resources/Census_clean_Midpoints_added.csv'
income_data = pd.read_csv(income_data_path)

# Display the first few rows of the income data
income_data.head()

In [ ]:
# Merging the combined top registrations data with the income data based on postcode
merged_data = pd.merge(combined_df, income_data, on='Postcode')

# Displaying the first few rows of the merged DataFrame
merged_data.head()


In [ ]:
#drop the extra state column
cols_to_drop = ['Total personal income (weekly)', 'Age', 'State_y']

merged_data = merged_data.drop(columns=cols_to_drop)

merged_data.head()

In [ ]:
#rename the remaining columns
cols_to_rename = {'State_x': 'State', 'Income_Midpoint': 'Income', 'Age_Midpoint': 'Age'}
merged_data = merged_data.rename(columns=cols_to_rename)
merged_data.head()

In [ ]:
#Save the new merged data ready for plotting manipulation
merged_data.to_csv('Resources/Top10_for_all_years_with_Census_cleaned.csv', index=False)